In [35]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

In [36]:
df_train=pd.read_csv("train.csv")
df_test=pd.read_csv("test.csv")
conversion_dict={0:'Real',1:'Fake'}
df_train['label']=df_train['label'].replace(conversion_dict)
df_train.label.value_counts()

Fake    10413
Real    10387
Name: label, dtype: int64

In [37]:
print(df_train.shape)
print(df_train.head,
df_test.head,
df_train['text'].isna().sum(),
df_test['text'].isna().sum())

(20800, 5)
<bound method NDFrame.head of           id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn  

In [38]:
df_train.dropna(axis=0,how="any",thresh=None,subset=['text'],inplace=True)
df_test.fillna(' ')

,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...
...,...,...,...,...
5195,25995,The Bangladeshi Traffic Jam That Never Ends - ...,Jody Rosen,Of all the dysfunctions that plague the world’...
5196,25996,John Kasich Signs One Abortion Bill in Ohio bu...,Sheryl Gay Stolberg,WASHINGTON — Gov. John Kasich of Ohio on Tu...
5197,25997,"California Today: What, Exactly, Is in Your Su...",Mike McPhate,Good morning. (Want to get California Today by...
5198,25998,300 US Marines To Be Deployed To Russian Borde...,,« Previous - Next » 300 US Marines To Be Deplo...


In [40]:
length=[]
for text in df_train['text']:
    length.append(len(str(text)))
df_train['length']=length
max_train=max(df_train['length'])
min_train=min(df_train['length'])
average=round((sum(df_train['length']))/len(df_train['length']))
print(max_train,min_train,average)
    

142961 1 4553


In [42]:
len(df_train[df_train['length']<50])

207

In [45]:
print(df_train['text'][df_train['length']<50])

82                                                   
169                                                  
173                                   Guest   Guest  
196            They got the heater turned up on high.
295                                                  
                             ...                     
20350                         I hope nobody got hurt!
20418                                 Guest   Guest  
20431    \nOctober 28, 2016 The Mothers by stclair by
20513                                                
20636                              Trump all the way!
Name: text, Length: 207, dtype: object


In [46]:
df_train=df_train.drop(df_train['text'][df_train['length']<50].index,axis=0)

In [47]:
max_train=max(df_train['length'])
min_train=min(df_train['length'])
average=round((sum(df_train['length']))/len(df_train['length']))
print(max_train,min_train,average)

142961 50 4598


In [53]:
train_labels=df_train['label']

x_train,x_test,y_train,y_test=train_test_split(df_train['text'],train_labels,test_size=0.1,random_state=7,shuffle=True)

In [57]:
tfidf=TfidfVectorizer(stop_words='english',max_df=0.7)
tfidf_train=tfidf.fit_transform(x_train)
tfidf_test=tfidf.transform(x_test)

In [61]:
pac=PassiveAggressiveClassifier(max_iter=50)
pac.fit(tfidf_train,y_train)
y_pred=pac.predict(tfidf_test)
score=accuracy_score(y_test,y_pred)
print(f'Accuracy: {round(score * 100, 2)}%')

Accuracy: 96.5%
